In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pycountry

In [ ]:
df_limpo= pd.read_csv('df_limpo.csv')
df_limpo.head()

In [ ]:
'''A função groupby é usada para agrupar dados com base em um critério
e aplicar operações em cada grupo separadamente.
groupby não altera o DataFrame original
ele cria um novo dataframe com o agrupamento'''

In [ ]:
'''sort_values é usado para ordenar um dataframe com base em uma coluna
ascending=True significa em ordem crescente. Por padrão, a ordem é crescente
ascending=False significa em ordem decrescente
inplace=True altera o DataFrame original
funciona tanto em DataFrame quanto em Series'''

In [ ]:
#aqui o groupby está agrupando pela experiencia (senior, pleno..)
#com o mean vai tirar a média do salario de cada grupo
#aqui sort_values está ordenando a média em ordem crescente
#index está dizendo que a variavel ordem só vai guardar os indices da ordenação
#esse indices são: junior, pleno, senior, executivo
ordem= ( df_limpo
        .groupby('experiencia')['salario_em_usd']
        .mean()
        .round(2)
        .sort_values(ascending=True)
        .index )

print(ordem)

In [ ]:
#criando um gráfico de barras com o salário médio em dolar de cada categoria de experiência
#order=ordem define como as barras vão ser ordenadas
#barras ordenadas de acordo com o indice criado anteriormente na variavel ordem
plt.figure(figsize=(8,5))
sns.barplot(data=df_limpo, x='experiencia', y='salario_em_usd', order=ordem)
plt.title('Salário médio por experiência')
plt.xlabel('Experiência')
plt.ylabel('Salário médio anual (USD)')
plt.show()

In [ ]:
#criando um histograma da distribuição de frequencia por faixa salarial
#bins=100 diz quantas "barras" vai ter
#kde=True/False é se vai ter a linha ou não
plt.figure(figsize=(8,4))
sns.histplot(df_limpo['salario_em_usd'], bins=100, kde=True, color='green')
plt.title('Distribuição de salários')
plt.xlabel('Salários em dolar')
plt.ylabel('Frequência')
plt.show()

In [ ]:
#x/y define a orientação do boxplot
plt.figure(figsize=(18,3))
sns.boxplot(x=df_limpo['salario_em_usd'], color='red')
plt.title('Distribuição de salários')
plt.xlabel('Salários em dolar')
plt.show()

In [ ]:
#aqui eu informo quais categorias vão ter um boxplot
#é uma lista com os valores em ordem que vou usar no parametro
ordem_exp=['Junior', 'Pleno', 'Senior', 'Executivo']

#criando um boxplot para cada categoria de experiencia
#hue e palette trabalham juntos para colorir cada box com uma cor diferente
#o hue informa que é para dividir e o palette informa qual cor vai ser cada
plt.figure(figsize=(8,8))
sns.boxplot(x='experiencia', y='salario_em_usd',
            data=df_limpo, order=ordem_exp,
            palette='Set2', hue='experiencia')

plt.title('Distribuição de salários por experiência', color='blue')
plt.xlabel('Categorias de experiência', color='blue')
plt.ylabel('Salário em dolar', color='blue')
plt.show()

In [ ]:
#criando um gráfico interativo com a biblioteca plotly
figura = px.bar(df_limpo.groupby('experiencia')['salario_em_usd'].mean().reset_index(),
             x='experiencia',
             y='salario_em_usd',
             #vai ordenar o eixo x(experiencia) de acordo com a lista ordem_exp
             category_orders={'experiencia': ordem_exp},
             #nomeando os eixos
             labels={'salario_em_usd': 'Salário médio (USD)', 'experiencia': 'Experiência'},
             title='Salário médio por nível de experiência')

figura.show()

In [ ]:
'''O reset_index() serve para recriar o índice. o antigo indice vira uma nova coluna
df.reset_index(drop=True) O índice antigo é apagado.
df.reset_index(drop=True, inplace=True) altera o dataframe original'''

In [ ]:
#com value_counts eu conto quantas pessoas com trabalho hibrido, presencial e remoto tem
#o reset_index vai resetar o indice da tabela da variavel remoto_contagem
remoto_contagem= df_limpo['tipo_trabalho'].value_counts().reset_index()
remoto_contagem.columns= ['tipo', 'quantidade'] #renomeando as colunas da tabela

#criando um gráfico de pizza com plotly
figura_pizza= px.pie(
    remoto_contagem,
    names='tipo',
    values='quantidade',
    title='Proporção dos tipos de trabalho'
)

figura_pizza.show()

In [ ]:
#faz o mesmo que o anterior
remoto_contagem= df_limpo['tipo_trabalho'].value_counts().reset_index()
remoto_contagem.columns= ['tipo', 'quantidade']

#criando um gráfico de rosca
figura_rosca= px.pie(
    remoto_contagem,
    names='tipo',
    values='quantidade',
    title='Proporção dos tipos de trabalho',
    #a diferença está aqui
    #o parametro hole define a largura do buraco no centro, assim vira uma rosca
    hole=0.5
)
#dizendo que dentro do gráfico é para mostra a porcentagem e o trabalho que ela se refere
figura_rosca.update_traces(textinfo='percent+label')
figura_rosca.show()

In [ ]:
'''A função apply() serve para aplicar uma função a dados de um DataFrame ou Series,
linha a linha ou coluna a coluna.
Você escolhe se a função vai rodar:
por linha (axis=1)
por coluna (axis=0) é o padrão'''

In [ ]:
#função para converter as abreviaturas dos paises de 2 letras para 3 letras
def converter_iso3(code):
    try:
        return pycountry.countries.get(alpha_2=code).alpha_3
    except:
        return None

# criando uma nova coluna no dataframe para a nova abreviação
# com a função apply eu aplico a função que criei na coluna residencia
df_limpo['residencia_iso3'] = df_limpo['residencia'].apply(converter_iso3)